# Scaffold for Recursive Emergence Detection via GEE + 8D ψ⁰ Field Analysis

---

### Motivation

The Amazon is vast, densely vegetated, and shaped by centuries of natural forces—leaving archaeological clues deeply obscured. Traditional detection methods often struggle due to:

* Limited high-resolution LiDAR coverage
* Noisy elevation patterns
* Insufficient labeled data for standard machine learning

But with **Recursive Emergence (RE)**, we flip the challenge into a signal: even faint structural patterns—repeating across space and scale—can hint at civilization.

We start with **windmills** as analogues to **Amazonian citadels**—recurring architectural motifs, small-scale but geometrically coherent. Our method builds from:

```
ψ⁰ (gradient field) → φ⁰ (coherence field) → G₂ (emergent kernel motif)
```

This is a data-efficient strategy that works even when machine learning fails.



In [ ]:
#@title Initialization (Numpy, Mathplot, Earth Engine, etc.)

import numpy as np
import matplotlib.pyplot as plt
from matplotlib import gridspec
import matplotlib.patches as patches
import folium
import ee
import geemap.foliumap as geemap
from typing import List, Dict, Tuple, Optional, Any
import logging
from dataclasses import dataclass, field
from scipy.ndimage import (
    uniform_filter, gaussian_filter, maximum_filter,
    binary_erosion, binary_dilation, binary_opening, label,
    distance_transform_edt
)
from scipy import ndimage
from scipy.stats import entropy, skew, kurtosis
from scipy.spatial import ConvexHull
from skimage.measure import regionprops, find_contours
from skimage.morphology import disk
import cv2
import time
import warnings
warnings.filterwarnings('ignore')

# Configure logging with enhanced formatting
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

# Initialize Earth Engine
try:
    ee.Initialize()
except:
    ee.Authenticate()
    ee.Initialize(project='amazon-discovery-research')  # Replace with your project ID

print("✅ Environment setup complete")

# Step 1: **Fetch patches in interest from Google Earth Engine (GEE)**

* Use known coordinates (e.g., windmill at Zaanse Schans)
* Patch size: customizable square (e.g., 40m × 40m for windmill, 100m × 100m for Amazon citadel)
* Extract raw elevation (AHN4 DSM)

In [ ]:
# Default training windmills for Dutch windmill detection (3 for ψ⁰ kernel construction)
DEFAULT_TRAINING_WINDMILLS = [
    {"name": "De Kat", "lat": 52.47505310183309, "lon": 4.8177388422949585},  # unchanged
    {"name": "De Zoeker", "lat": 52.47590104112108, "lon": 4.817647238879872},  # moved 5m north
    {"name": "Het Jonge Schaap", "lat": 52.47621811347626, "lon": 4.816644787814995}  # moved 5m south + 5m west
]

# Define validation sites
DEFAULT_VALIDATION_WINDMILLS = [
    {"name": "De Bonte Hen", "lat": 52.47793734015221, "lon": 4.813402499137949},
    {"name": "De Gekroonde Poelenburg", "lat": 52.474166977199445, "lon": 4.817628676751737},
    {"name": "De Huisman", "lat": 52.47323132365517, "lon": 4.816668420518732},
    {"name": "Het Klaverblad", "lat": 52.4775485810242, "lon": 4.813724798553969},
    {"name": "Control_Bonte_Hen_80m_east", "lat": 52.47793734015221, "lon": 4.814542499137949},
    {"name": "Control_Poelenburg_80m_east", "lat": 52.474166977199445, "lon": 4.818768676751737},
    {"name": "Control_Huisman_80m_east", "lat": 52.47323132365517, "lon": 4.817808420518732},
    {"name": "Control_Klaverblad_80m_east", "lat": 52.4775485810242, "lon": 4.814864798553969},
]

### Step 4: φ⁰ Collapse Projection and Ω–SIM Diagnostics

This section visualizes windmill sites with φ⁰ projection diagnostics using the Ω–SIM Axiomatic Framework. 
- ψ⁰ contradiction norm: radial symmetry deviation
- τ: curl of slope field (torsion)
- ΔΛ: spectral variance of FFT
- Σ: gradient field variance
- Entropy: histogram distribution

All overlays and metrics are rendered on an interactive Earth Engine map, ready for Colab and Kaggle.

In [ ]:
# Install required packages if running in Colab
import sys
if 'google.colab' in sys.modules:
    !pip install -q geemap earthengine-api folium

import ee
import geemap
import folium
import numpy as np

# Authenticate and initialize Earth Engine
try:
    ee.Initialize()
except Exception:
    ee.Authenticate()
    ee.Initialize()

def compute_phi0_metrics(dsm_arr):
    """
    Compute Ω–SIM φ⁰ diagnostics for a DSM patch.
    - ψ⁰ contradiction: radial symmetry deviation
    - τ: curl-like torsion in slope field
    - ΔΛ: spectral variance of FFT
    - Σ: gradient field variance
    - Entropy: histogram entropy
    """
    center = np.array(dsm_arr.shape) // 2
    y, x = np.indices(dsm_arr.shape)
    r = np.sqrt((x - center[1])**2 + (y - center[0])**2)
    radial_profile = np.array([dsm_arr[r.astype(int) == i].mean() for i in range(1, min(center))])
    psi0_contradiction = np.std(radial_profile) / (np.mean(radial_profile) + 1e-6)

    gy, gx = np.gradient(dsm_arr)
    curl = gx - gy
    tau = np.mean(np.abs(curl))

    fft = np.fft.fft2(dsm_arr)
    fft_shift = np.fft.fftshift(fft)
    spectrum = np.abs(fft_shift)
    delta_lambda = np.std(spectrum) / (np.mean(spectrum) + 1e-6)

    grad_mag = np.sqrt(gx**2 + gy**2)
    sigma = np.var(grad_mag)

    hist, _ = np.histogram(dsm_arr, bins=32, density=True)
    entropy_val = -np.sum(hist * np.log(hist + 1e-8))

    return {
        "ψ⁰ contradiction": psi0_contradiction,
        "τ (torsion)": tau,
        "ΔΛ (spectral divergence)": delta_lambda,
        "Σ score": sigma,
        "Entropy": entropy_val
    }

def show_site_on_map(sites, zoom=18):
    """
    Visualize windmill sites with φ⁰ projection diagnostics using Ω–SIM framework.
    - Clips AHN4 DSM, draws contours, lines, and overlays φ⁰ metrics.
    - Adds HTML popups with all diagnostics for each site.
    - Designed for Google Colab + Earth Engine.
    """
    m = geemap.Map(center=(sites[0]['lat'], sites[0]['lon']), zoom=zoom)

    for site in sites:
        # 1. Define geometry and clip DSM
        center = ee.Geometry.Point([site['lon'], site['lat']])
        patch = center.buffer(50).bounds()  # 100m x 100m
        dsm = ee.Image("AHN/AHN4").select('dsm').clip(patch)

        # 2. Generate contours
        contour_interval = 0.5
        dsm_scaled = dsm.divide(contour_interval).floor().multiply(contour_interval).toInt()
        contours = dsm_scaled.reduceToVectors(
            geometry=patch,
            geometryType='polygon',
            scale=0.5,
            maxPixels=1e10
        )

        # 3. Draw A–B and C–D lines through center
        offset = 0.00045  # ~50m
        ab_line = ee.Geometry.LineString([
            [site['lon'] - offset, site['lat'] - offset],
            [site['lon'] + offset, site['lat'] + offset]
        ])
        cd_line = ee.Geometry.LineString([
            [site['lon'] + offset, site['lat'] - offset],
            [site['lon'] - offset, site['lat'] + offset]
        ])

        # 4. Download DSM patch as numpy array for diagnostics
        url = dsm.getDownloadURL({
            'scale': 1,
            'region': patch,
            'format': 'NPY'
        })
        import requests, io
        arr = np.load(io.BytesIO(requests.get(url).content))
        arr = np.nan_to_num(arr)

        # 5. Compute φ⁰ diagnostics
        metrics = compute_phi0_metrics(arr)

        # 6. Style overlays
        dsm_vis = {
            'min': float(np.nanmin(arr)),
            'max': float(np.nanmax(arr)),
            'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']
        }
        outline_color = 'green' if metrics["ψ⁰ contradiction"] < 0.15 else 'red'

        # 7. Add layers to map
        m.addLayer(dsm, dsm_vis, f"DSM: {site['name']}")
        m.addLayer(contours, {'color': 'black'}, f"Contours: {site['name']}")
        m.addLayer(ab_line, {'color': 'blue', 'width': 3}, f"A–B: {site['name']}")
        m.addLayer(cd_line, {'color': 'purple', 'width': 3}, f"C–D: {site['name']}")
        m.addLayer(patch, {'color': outline_color, 'fillOpacity': 0}, f"Outline: {site['name']}")

        # 8. Add HTML popup with φ⁰ metrics
        html = f"""
        <b>{site['name']}</b><br>
        <ul>
        <li>ψ⁰ contradiction: {metrics['ψ⁰ contradiction']:.3f}</li>
        <li>τ (torsion): {metrics['τ (torsion)']:.3f}</li>
        <li>ΔΛ (spectral divergence): {metrics['ΔΛ (spectral divergence)']:.3f}</li>
        <li>Σ score: {metrics['Σ score']:.3f}</li>
        <li>Entropy: {metrics['Entropy']:.3f}</li>
        </ul>
        """
        folium.Marker(
            location=[site['lat'], site['lon']],
            popup=folium.Popup(html, max_width=350),
            icon=folium.Icon(color='green' if outline_color == 'green' else 'red')
        ).add_to(m)

    m.add_html("<b>Ω–SIM φ⁰ Projection Map (Windmill Sites)</b>", position='topright')
    display(m)

In [ ]:
# Test call for Step 4: φ⁰ projection map
show_site_on_map(DEFAULT_TRAINING_WINDMILLS)